In [1]:
# %pip install --upgrade pip
# %pip install -r requirements.txt

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import IsolationForest

In [2]:
# load data
data = pd.read_csv('./dataset/artificialWithAnomaly/artificialWithAnomaly/art_load_balancer_spikes.csv')
data.tail(10)

,timestamp,value
4022,2014-04-14 23:10:00,0.197762
4023,2014-04-14 23:15:00,0.000000
4024,2014-04-14 23:20:00,0.000000
4025,2014-04-14 23:25:00,0.000000
4026,2014-04-14 23:30:00,0.000000
4027,2014-04-14 23:35:00,0.460912
4028,2014-04-14 23:40:00,0.472003
4029,2014-04-14 23:45:00,0.000000
4030,2014-04-14 23:50:00,0.000000
4031,2014-04-14 23:55:00,0.000000


In [3]:
# define and fit the model
random_state = np.random.RandomState(42)
model = IsolationForest(n_estimators=100,
                        max_samples='auto',
                        contamination=float(0.15),
                        random_state=random_state,
                        warm_start=False)
model.fit(data[['value']])

print(model.get_params())

{'bootstrap': False, 'contamination': 0.15, 'max_features': 1.0, 'max_samples': 'auto', 'n_estimators': 100, 'n_jobs': None, 'random_state': RandomState(MT19937) at 0x177129E40, 'verbose': 0, 'warm_start': False}


/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [4]:
# get anomaly scores
data['scores'] = model.decision_function(data[['value']])
data['anomaly_score'] = model.predict(data[['value']])
data[data['anomaly_score']==-1].head()

,timestamp,value,scores,anomaly_score
69,2014-04-01 05:45:00,0.146560,-0.203659,-1
70,2014-04-01 05:50:00,0.154264,-0.199369,-1
98,2014-04-01 08:10:00,0.249212,-0.227582,-1
99,2014-04-01 08:15:00,0.225824,-0.209279,-1
154,2014-04-01 12:50:00,0.128654,-0.217484,-1


In [5]:
# evaluate the model
anomaly_count = 455
accuracy = 100*list(data['anomaly_score']).count(-1)/anomaly_count
print('Model accuracy: ', accuracy)


Model accuracy:  100.0
